#Imports

In [ ]:
!pip install scikeras
!pip install tabulate
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,cohen_kappa_score, precision_score, f1_score,recall_score,make_scorer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tabulate import tabulate

#Data preprocessing

In [ ]:
file_names = ['amazon_cells_labelled.txt', 'imdb_labelled.txt', 'yelp_labelled.txt']

# Cargar datos desde múltiples archivos
dfs = []
for file_name in file_names:
    file_path = f"{file_name}"
    df = pd.read_csv(file_path, sep='\t', header=None, names=['sentence', 'sentiment'])
    dfs.append(df)

# Concatenar los marcos de datos
df = pd.concat(dfs, ignore_index=True)

# Barajar el conjunto de datos
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Descargar recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('punkt')

# Definir palabras de parada en inglés
stop_words = set(stopwords.words('english'))

# Imprimir el conjunto de datos original
print("ANTES")
print(df)

# Función para preprocesar el texto
def preprocess_text(text):
    words = word_tokenize(text.lower())
    filtered_words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(filtered_words)

# Aplicar la función de preprocesamiento al conjunto de datos
df['processed_sentence'] = df['sentence'].apply(preprocess_text)

# Imprimir el conjunto de datos después del preprocesamiento
print("DESPUÉS")
print(df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


BEFORE
                                               sentence  sentiment
0     It's close to my house, it's low-key, non-fanc...          1
1     If you stay in Vegas you must get breakfast he...          1
2     Let's start with all the problemsthe acting, ...          0
3     It's too bad that everyone else involved didn'...          0
4     i felt insulted and disrespected, how could yo...          0
...                                                 ...        ...
2743  Lot of holes in the plot: there's nothing abou...          0
2744     Worst hour and a half of my life!Oh my gosh!            0
2745  It's also great to see that renowned silent sc...          1
2746                 But this movie really got to me.            1
2747                         Last time buying from you.          0

[2748 rows x 2 columns]
AFTER
                                               sentence  sentiment  \
0     It's close to my house, it's low-key, non-fanc...          1   
1     If you stay 

# Model implementations

#Dummy

In [ ]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['processed_sentence'], df['sentiment'], test_size=0.2, random_state=42)

# Crear un clasificador ficticio (Dummy Classifier) utilizando CountVectorizer
dummy_clf = make_pipeline(CountVectorizer(), DummyClassifier(strategy='most_frequent'))

# Entrenar el clasificador ficticio con el conjunto de entrenamiento
dummy_clf.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_dummy = dummy_clf.predict(X_test)

# Calcular métricas de desempeño del clasificador ficticio
accuracy_dummy = accuracy_score(y_test, y_pred_dummy)
precision_dummy = precision_score(y_test, y_pred_dummy)
recall_dummy = recall_score(y_test, y_pred_dummy)
f1_dummy = f1_score(y_test, y_pred_dummy)

# Imprimir el desempeño del clasificador ficticio
print("Desempeño del Dummy Classifier:")
print(f"Exactitud (Accuracy): {accuracy_dummy}")
print(f"Precisión (Precision): {precision_dummy}")
print(f"Recall: {recall_dummy}")
print(f"F1-score: {f1_dummy}")
print("Predicciones del Dummy Classifier:")
print(y_pred_dummy)

DummyClassifier Performance:
Accuracy: 0.5018181818181818
Precision: 0.5018181818181818
Recall: 1.0
F1-score: 0.6682808716707022
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

#RNN

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['processed_sentence'], df['sentiment'], test_size=0.2, random_state=42)

# Parámetros para la tokenización y secuencia de texto
max_words = 10000
max_len = 100

# Tokenizar y convertir texto a secuencias
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences para igualar la longitud de las secuencias
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Codificar las etiquetas de clase
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Construir el modelo de red neuronal recurrente (RNN)
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SimpleRNN(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con el conjunto de entrenamiento
history = model.fit(X_train_pad, y_train_encoded, validation_split=0.2,
                    epochs=25, batch_size=128, callbacks=[EarlyStopping(patience=3)])

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_pad)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calcular métricas de desempeño del modelo RNN
accuracy_rnn = accuracy_score(y_test_encoded, y_pred_binary)
precision_rnn = precision_score(y_test_encoded, y_pred_binary)
recall_rnn = recall_score(y_test_encoded, y_pred_binary)
f1_rnn = f1_score(y_test_encoded, y_pred_binary)
kappa_rnn = cohen_kappa_score(y_test_encoded, y_pred_binary)

# Imprimir el desempeño del modelo RNN
print("Desempeño del modelo RNN:")
print(f"Exactitud (Accuracy): {accuracy_rnn}")
print(f"Precisión (Precision): {precision_rnn}")
print(f"Recall: {recall_rnn}")
print(f"Kappa-score: {kappa_rnn}")
print(f"F1-score: {f1_rnn}")
#print(y_pred_binary)

Epoch 1/25
14/14 [==============================] - 3s 87ms/step - loss: 0.6989 - accuracy: 0.5171 - val_loss: 0.6851 - val_accuracy: 0.5500
Epoch 2/25
14/14 [==============================] - 1s 99ms/step - loss: 0.6329 - accuracy: 0.7002 - val_loss: 0.6638 - val_accuracy: 0.6250
Epoch 3/25
14/14 [==============================] - 1s 100ms/step - loss: 0.5356 - accuracy: 0.8663 - val_loss: 0.6443 - val_accuracy: 0.6182
Epoch 4/25
14/14 [==============================] - 1s 97ms/step - loss: 0.3970 - accuracy: 0.9329 - val_loss: 0.5836 - val_accuracy: 0.6955
Epoch 5/25
14/14 [==============================] - 1s 73ms/step - loss: 0.2501 - accuracy: 0.9545 - val_loss: 0.5350 - val_accuracy: 0.7432
Epoch 6/25
14/14 [==============================] - 1s 102ms/step - loss: 0.1545 - accuracy: 0.9710 - val_loss: 0.5189 - val_accuracy: 0.7295
Epoch 7/25
14/14 [==============================] - 1s 104ms/step - loss: 0.1022 - accuracy: 0.9761 - val_loss: 0.5546 - val_accuracy: 0.7136
Epoch 8/25

#Best Params for RNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder

# Función para construir el modelo
def build_rnn_model(optimizer="adam", loss="binary_crossentropy", batch_size=128, epochs=25):
    model = Sequential()
    model.add(Embedding(max_words, 128, input_length=max_len))
    model.add(SimpleRNN(64))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

# Crear el clasificador Keras para GridSearchCV
rnn_classifier = KerasClassifier(build_fn=build_rnn_model, verbose=0)

# Definir los parámetros a buscar
param_grid = {
    'optimizer': ["adam", "sgd", "rmsprop"],
    'loss': ["binary_crossentropy"],
    'batch_size': [32, 64, 128],
    'epochs': [25, 50],
}

# Crear el objeto GridSearchCV
grid = GridSearchCV(estimator=rnn_classifier, param_grid=param_grid, scoring='accuracy', cv=3)

# Preprocesar los datos nuevamente para asegurarse de que todo está en orden
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

y_train_encoded = label_encoder.fit_transform(y_train)

# Ejecutar la búsqueda de hiperparámetros
grid_result = grid.fit(X_train_pad, y_train_encoded)

# Imprimir los resultados
print(f"Mejor Exactitud: {grid_result.best_score_} utilizando {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Exactitud: {mean} (±{stdev}) con: {param}")


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras

Best Accuracy: 0.9330775041809863 using {'batch_size': 128, 'epochs': 25, 'loss': 'binary_crossentropy', 'optimizer': 'rmsprop'}
Accuracy: 0.9044169356662368 (±0.06494731402456407) with: {'batch_size': 32, 'epochs': 25, 'loss': 'binary_crossentropy', 'optimizer': 'adam'}
Accuracy: 0.8602867175094492 (±0.06477390474258453) with: {'batch_size': 32, 'epochs': 25, 'loss': 'binary_crossentropy', 'optimizer': 'sgd'}
Accuracy: 0.9003129837954162 (±0.07930996313486716) with: {'batch_size': 32, 'epochs': 25, 'loss': 'binary_crossentropy', 'optimizer': 'rmsprop'}
Accuracy: 0.8452724412736042 (±0.07067627162023384) with: {'batch_size': 32, 'epochs': 50, 'loss': 'binary_crossentropy', 'optimizer': 'adam'}
Accuracy: 0.8757526396747651 (±0.06799795444379347) with: {'batch_size': 32, 'epochs': 50, 'loss': 'binary_crossentropy', 'optimizer': 'sgd'}
Accuracy: 0.8707478809294836 (±0.06988584171544238) with: {'batch_size': 32, 'epochs': 50, 'loss': 'binary_crossentropy', 'optimizer': 'rmsprop'}
Accuracy:

In [ ]:
# Imprimir un resumen del modelo RNN
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 100, 128)          1280000   
                                                                 
 simple_rnn_12 (SimpleRNN)   (None, 64)                12352     
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1292417 (4.93 MB)
Trainable params: 1292417 (4.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


#RNN with best params

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['processed_sentence'], df['sentiment'], test_size=0.2, random_state=42)

# Parámetros para la tokenización y secuencia de texto
max_words = 10000
max_len = 100

# Tokenizar y convertir texto a secuencias
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences para igualar la longitud de las secuencias
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Codificar las etiquetas de clase
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Construir el modelo de red neuronal recurrente (RNN)
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SimpleRNN(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con el conjunto de entrenamiento
history = model.fit(X_train_pad, y_train_encoded, validation_split=0.2,
                    epochs=25, batch_size=128, callbacks=[EarlyStopping(patience=3)])

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_pad)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calcular métricas de desempeño del modelo RNN
accuracy_rnn = accuracy_score(y_test_encoded, y_pred_binary)
precision_rnn = precision_score(y_test_encoded, y_pred_binary)
recall_rnn = recall_score(y_test_encoded, y_pred_binary)
f1_rnn = f1_score(y_test_encoded, y_pred_binary)
kappa_rnn = cohen_kappa_score(y_test_encoded, y_pred_binary)

# Imprimir el desempeño del modelo RNN
print("Desempeño del modelo RNN:")
print(f"Exactitud (Accuracy): {accuracy_rnn}")
print(f"Precisión (Precision): {precision_rnn}")
print(f"Recall: {recall_rnn}")
print(f"F1-score: {f1_rnn}")
print(f"Kappa-score: {kappa_rnn}")

#LSTM

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['processed_sentence'], df['sentiment'], test_size=0.2, random_state=42)

# Parámetros para la tokenización y secuencia de texto
max_words = 10000
max_len = 100

# Tokenizar y convertir texto a secuencias
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences para igualar la longitud de las secuencias
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Codificar las etiquetas de clase
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Construir el modelo LSTM
model_lstm = Sequential()
model_lstm.add(Embedding(max_words, 128, input_length=max_len))
model_lstm.add(LSTM(64))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con el conjunto de entrenamiento
history = model_lstm.fit(X_train_pad, y_train_encoded, validation_split=0.2,
                         epochs=25, batch_size=128, callbacks=[EarlyStopping(patience=3)])

# Realizar predicciones en el conjunto de prueba
y_pred = model_lstm.predict(X_test_pad)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calcular métricas de desempeño del modelo LSTM
accuracy_lstm = accuracy_score(y_test_encoded, y_pred_binary)
precision_lstm = precision_score(y_test_encoded, y_pred_binary)
recall_lstm = recall_score(y_test_encoded, y_pred_binary)
f1_lstm = f1_score(y_test_encoded, y_pred_binary)
kappa_lstm = cohen_kappa_score(y_test_encoded, y_pred_binary)

# Imprimir el desempeño del modelo LSTM
print("Desempeño del modelo LSTM:")
print(f"Exactitud (Accuracy): {accuracy_lstm}")
print(f"Precisión (Precision): {precision_lstm}")
print(f"Recall: {recall_lstm}")
print(f"F1-score: {f1_lstm}")
print(f"Kappa-score: {kappa_lstm}")

Epoch 1/25
14/14 [==============================] - 13s 276ms/step - loss: 0.6916 - accuracy: 0.5233 - val_loss: 0.6863 - val_accuracy: 0.6477
Epoch 2/25
14/14 [==============================] - 3s 240ms/step - loss: 0.6724 - accuracy: 0.7622 - val_loss: 0.6674 - val_accuracy: 0.7273
Epoch 3/25
14/14 [==============================] - 5s 383ms/step - loss: 0.6114 - accuracy: 0.8350 - val_loss: 0.6121 - val_accuracy: 0.6341
Epoch 4/25
14/14 [==============================] - 3s 242ms/step - loss: 0.4555 - accuracy: 0.8584 - val_loss: 0.5200 - val_accuracy: 0.7636
Epoch 5/25
14/14 [==============================] - 3s 235ms/step - loss: 0.3175 - accuracy: 0.8965 - val_loss: 0.4578 - val_accuracy: 0.8114
Epoch 6/25
14/14 [==============================] - 4s 282ms/step - loss: 0.2131 - accuracy: 0.9448 - val_loss: 0.4864 - val_accuracy: 0.8000
Epoch 7/25
14/14 [==============================] - 5s 337ms/step - loss: 0.1497 - accuracy: 0.9534 - val_loss: 0.5038 - val_accuracy: 0.7955
Epoch

#Best params for LSTM

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder

# Función para construir el modelo LSTM
def build_lstm_model(optimizer="adam", loss="binary_crossentropy"):
    model_lstm = Sequential()
    model_lstm.add(Embedding(max_words, 128, input_length=max_len))
    model_lstm.add(LSTM(64))
    model_lstm.add(Dropout(0.5))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model_lstm

# Crear el clasificador Keras para GridSearchCV
lstm_classifier = KerasClassifier(build_fn=build_lstm_model, epochs=25, batch_size=128, verbose=0)

# Definir los parámetros a buscar
param_grid = {
    'optimizer': ["adam", "sgd", "rmsprop"],
    'loss': ["binary_crossentropy"],
}

# Crear el objeto GridSearchCV
grid = GridSearchCV(estimator=lstm_classifier, param_grid=param_grid, scoring='accuracy', cv=3)

# Preprocesar los datos nuevamente para asegurarse de que todo está en orden
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

y_train_encoded = label_encoder.fit_transform(y_train)

# Ejecutar la búsqueda de hiperparámetros
grid_result = grid.fit(X_train_pad, y_train_encoded)

# Imprimir los resultados
print(f"Mejor Exactitud: {grid_result.best_score_} utilizando {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Exactitud: {mean} (±{stdev}) con: {param}")


#LSTM with best params

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['processed_sentence'], df['sentiment'], test_size=0.2, random_state=42)

# Parámetros para la tokenización y secuencia de texto
max_words = 10000
max_len = 100

# Tokenizar y convertir texto a secuencias
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences para igualar la longitud de las secuencias
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Codificar las etiquetas de clase
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Construir el modelo LSTM
model_lstm = Sequential()
model_lstm.add(Embedding(max_words, 128, input_length=max_len))
model_lstm.add(LSTM(64))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con el conjunto de entrenamiento
history = model_lstm.fit(X_train_pad, y_train_encoded, validation_split=0.2,
                         epochs=25, batch_size=128, callbacks=[EarlyStopping(patience=3)])

# Realizar predicciones en el conjunto de prueba
y_pred = model_lstm.predict(X_test_pad)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calcular métricas de desempeño del modelo LSTM
accuracy_lstm = accuracy_score(y_test_encoded, y_pred_binary)
precision_lstm = precision_score(y_test_encoded, y_pred_binary)
recall_lstm = recall_score(y_test_encoded, y_pred_binary)
f1_lstm = f1_score(y_test_encoded, y_pred_binary)
kappa_lstm = cohen_kappa_score(y_test_encoded, y_pred_binary)

# Imprimir el desempeño del modelo LSTM
print("Desempeño del modelo LSTM:")
print(f"Exactitud (Accuracy): {accuracy_lstm}")
print(f"Precisión (Precision): {precision_lstm}")
print(f"Recall: {recall_lstm}")
print(f"F1-score: {f1_lstm}")
print(f"Kappa-score: {kappa_lstm}")
print(y_pred_binary)

Epoch 1/25
14/14 [==============================] - 9s 370ms/step - loss: 0.6916 - accuracy: 0.5250 - val_loss: 0.6877 - val_accuracy: 0.6455
Epoch 2/25
14/14 [==============================] - 5s 335ms/step - loss: 0.6766 - accuracy: 0.7452 - val_loss: 0.6725 - val_accuracy: 0.7545
Epoch 3/25
14/14 [==============================] - 5s 379ms/step - loss: 0.6267 - accuracy: 0.8584 - val_loss: 0.6123 - val_accuracy: 0.7182
Epoch 4/25
14/14 [==============================] - 4s 253ms/step - loss: 0.5207 - accuracy: 0.8248 - val_loss: 0.5501 - val_accuracy: 0.7841
Epoch 5/25
14/14 [==============================] - 3s 251ms/step - loss: 0.3911 - accuracy: 0.9135 - val_loss: 0.5060 - val_accuracy: 0.7705
Epoch 6/25
14/14 [==============================] - 5s 365ms/step - loss: 0.2891 - accuracy: 0.9300 - val_loss: 0.4345 - val_accuracy: 0.8091
Epoch 7/25
14/14 [==============================] - 4s 294ms/step - loss: 0.1908 - accuracy: 0.9511 - val_loss: 0.4619 - val_accuracy: 0.8045
Epoch 

#Definited functions

In [ ]:
#Data
X_train, X_test, y_train, y_test = train_test_split(df['processed_sentence'], df['sentiment'], test_size=0.2, random_state=42)

In [ ]:
#Dummy

def Dummy(X_train, X_test, y_train, y_test):

    metrics = {}

    # Crear un clasificador Dummy con la estrategia de predecir la clase más frecuente
    dummy_clf = make_pipeline(CountVectorizer(), DummyClassifier(strategy='most_frequent'))

    # Ajustar el clasificador Dummy al conjunto de entrenamiento
    dummy_clf.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred_dummy = dummy_clf.predict(X_test)

    # Calcular métricas de evaluación
    accuracy_dummy = accuracy_score(y_test, y_pred_dummy)
    precision_dummy = precision_score(y_test, y_pred_dummy)
    recall_dummy = recall_score(y_test, y_pred_dummy)
    f1_dummy = f1_score(y_test, y_pred_dummy)

    # Almacenar las métricas en un diccionario
    metrics['accuracy'] = accuracy_dummy
    metrics['precision'] = precision_dummy
    metrics['recall'] = recall_dummy
    metrics['f1'] = f1_dummy

    return metrics

In [ ]:
#RNN

def RNN(X_train, X_test, y_train, y_test):

    metrics = {}

    # Parámetros para la tokenización y secuencia de texto
    max_words = 10000
    max_len = 100

    # Tokenizar y convertir texto a secuencias
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)

    # Pad sequences para igualar la longitud de las secuencias
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

    # Codificar las etiquetas de clase
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Construir el modelo RNN
    model = Sequential()
    model.add(Embedding(max_words, 128, input_length=max_len))
    model.add(SimpleRNN(64))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compilar el modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo con el conjunto de entrenamiento
    history = model.fit(X_train_pad, y_train_encoded, validation_split=0.2,
                        epochs=25, batch_size=128)

    # Realizar predicciones en el conjunto de prueba
    y_pred = model.predict(X_test_pad)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calcular métricas de desempeño del modelo RNN
    accuracy_rnn = accuracy_score(y_test_encoded, y_pred_binary)
    precision_rnn = precision_score(y_test_encoded, y_pred_binary)
    recall_rnn = recall_score(y_test_encoded, y_pred_binary)
    f1_rnn = f1_score(y_test_encoded, y_pred_binary)

    # Almacenar las métricas en un diccionario
    metrics['accuracy'] = accuracy_rnn
    metrics['precision'] = precision_rnn
    metrics['recall'] = recall_rnn
    metrics['f1'] = f1_rnn

    return metrics

In [ ]:
#RNN Best Hyperparams

def RNN_best_hyperparams(X_train, X_test, y_train, y_test):

    metrics = {}

    # Parámetros para la tokenización y secuencia de texto
    max_words = 10000
    max_len = 100

    # Tokenizar y convertir texto a secuencias
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)

    # Pad sequences para igualar la longitud de las secuencias
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

    # Codificar las etiquetas de clase
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Construir el modelo RNN con hiperparámetros optimizados
    model = Sequential()
    model.add(Embedding(max_words, 128, input_length=max_len))
    model.add(SimpleRNN(64))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compilar el modelo
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo con el conjunto de entrenamiento
    history = model.fit(X_train_pad, y_train_encoded, validation_split=0.2,
                        epochs=25, batch_size=128)

    # Realizar predicciones en el conjunto de prueba
    y_pred = model.predict(X_test_pad)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calcular métricas de desempeño del modelo RNN con mejores hiperparámetros
    accuracy_rnn = accuracy_score(y_test_encoded, y_pred_binary)
    precision_rnn = precision_score(y_test_encoded, y_pred_binary)
    recall_rnn = recall_score(y_test_encoded, y_pred_binary)
    f1_rnn = f1_score(y_test_encoded, y_pred_binary)
    kappa_rnn = cohen_kappa_score(y_test_encoded, y_pred_binary)

    # Almacenar las métricas en un diccionario
    metrics['accuracy'] = accuracy_rnn
    metrics['precision'] = precision_rnn
    metrics['recall'] = recall_rnn
    metrics['f1'] = f1_rnn
    metrics['kappa'] = kappa_rnn

    return metrics

In [ ]:
#LSTM

def LSTMFunction(X_train, X_test, y_train, y_test):

    metrics = {}

    # Parámetros para la tokenización y secuencia de texto
    max_words = 10000
    max_len = 100

    # Tokenizar y convertir texto a secuencias
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)

    # Pad sequences para igualar la longitud de las secuencias
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

    # Codificar las etiquetas de clase
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Construir el modelo LSTM
    model_lstm = Sequential()
    model_lstm.add(Embedding(max_words, 128, input_length=max_len))
    model_lstm.add(LSTM(64))
    model_lstm.add(Dropout(0.5))
    model_lstm.add(Dense(1, activation='sigmoid'))

    # Compilar el modelo
    model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo con el conjunto de entrenamiento
    history = model_lstm.fit(X_train_pad, y_train_encoded, validation_split=0.2,
                             epochs=25, batch_size=128)

    # Realizar predicciones en el conjunto de prueba
    y_pred = model_lstm.predict(X_test_pad)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calcular métricas de desempeño del modelo LSTM
    accuracy_lstm = accuracy_score(y_test_encoded, y_pred_binary)
    precision_lstm = precision_score(y_test_encoded, y_pred_binary)
    recall_lstm = recall_score(y_test_encoded, y_pred_binary)
    f1_lstm = f1_score(y_test_encoded, y_pred_binary)
    kappa_lstm = cohen_kappa_score(y_test_encoded, y_pred_binary)

    # Almacenar las métricas en un diccionario
    metrics['accuracy'] = accuracy_lstm
    metrics['precision'] = precision_lstm
    metrics['recall'] = recall_lstm
    metrics['f1'] = f1_lstm
    metrics['kappa'] = kappa_lstm

    return metrics

In [ ]:
#LSTM Best Hyperparams

def LSTM_best_hyperparams(X_train, X_test, y_train, y_test):

    metrics = {}

    # Parámetros para la tokenización y secuencia de texto
    max_words = 10000
    max_len = 100

    # Tokenizar y convertir texto a secuencias
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)

    # Pad sequences para igualar la longitud de las secuencias
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

    # Codificar las etiquetas de clase
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Construir el modelo LSTM con hiperparámetros optimizados
    model_lstm = Sequential()
    model_lstm.add(Embedding(max_words, 128, input_length=max_len))
    model_lstm.add(LSTM(64))
    model_lstm.add(Dropout(0.5))
    model_lstm.add(Dense(1, activation='sigmoid'))

    # Compilar el modelo
    model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo con el conjunto de entrenamiento
    history = model_lstm.fit(X_train_pad, y_train_encoded, validation_split=0.2,
                             epochs=25, batch_size=128)

    # Realizar predicciones en el conjunto de prueba
    y_pred = model_lstm.predict(X_test_pad)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calcular métricas de desempeño del modelo LSTM con mejores hiperparámetros
    accuracy_lstm = accuracy_score(y_test_encoded, y_pred_binary)
    precision_lstm = precision_score(y_test_encoded, y_pred_binary)
    recall_lstm = recall_score(y_test_encoded, y_pred_binary)
    f1_lstm = f1_score(y_test_encoded, y_pred_binary)
    kappa_lstm = cohen_kappa_score(y_test_encoded, y_pred_binary)

    # Almacenar las métricas en un diccionario
    metrics['accuracy'] = accuracy_lstm
    metrics['precision'] = precision_lstm
    metrics['recall'] = recall_lstm
    metrics['f1'] = f1_lstm
    metrics['kappa'] = kappa_lstm

    return metrics

#Average


In [ ]:
def AverageStatisticsForModels(n, functionName, f, X_train, X_test, y_train, y_test):

    dictReturned = {}
    metrics = {}
    accuracy = 0.0
    precision = 0.0
    recall = 0.0
    f1 = 0.0
    kappa = 0.0

    # Realizar múltiples ejecuciones del modelo
    for i in range(n):
        metrics = f(X_train, X_test, y_train, y_test)
        accuracy += metrics['accuracy']
        precision += metrics['precision']
        recall += metrics['recall']
        f1 += metrics['f1']
        if 'kappa' in metrics:
            kappa += metrics['kappa']

    # Calcular promedio de las métricas
    accuracy = accuracy / n
    precision = precision / n
    recall = recall / n
    f1 = f1 / n
    kappa = kappa / n

    # Almacenar las métricas promedio en un diccionario
    dictReturned['Metrics'] = ['Accuracy', 'Precision', 'Recall', 'F1', 'Kappa_Cohen']
    dictReturned[f'{functionName}'] = [accuracy, precision, recall, f1, kappa]

    return dictReturned


#Stadistics for models

In [ ]:
n = 100

In [ ]:
dummyTabulable = AverageStadisticsForModels(n, 'Dummy Classifier', Dummy, X_train, X_test, y_train, y_test)
print(tabulate(dummyTabulable, headers='keys', tablefmt='fancy_grid'))

╒═════════════╤════════════════════╕
│ Metrics     │   Dummy Classifier │
╞═════════════╪════════════════════╡
│ Accuracy    │           0.501818 │
├─────────────┼────────────────────┤
│ Precision   │           0.501818 │
├─────────────┼────────────────────┤
│ Recall      │           1        │
├─────────────┼────────────────────┤
│ F1          │           0.668281 │
├─────────────┼────────────────────┤
│ Kappa_Cohen │           0        │
╘═════════════╧════════════════════╛


In [ ]:
RNNTabulable = AverageStadisticsForModels(n, 'RNN', RNN, X_train, X_test, y_train, y_test)
print(tabulate(RNNTabulable, headers='keys', tablefmt='fancy_grid'))

Streaming output truncated to the last 5000 lines.
14/14 [==============================] - 2s 114ms/step - loss: 0.1869 - accuracy: 0.9676 - val_loss: 0.6080 - val_accuracy: 0.6455
Epoch 7/25
14/14 [==============================] - 2s 123ms/step - loss: 0.1315 - accuracy: 0.9790 - val_loss: 0.5854 - val_accuracy: 0.7045
Epoch 8/25
14/14 [==============================] - 2s 118ms/step - loss: 0.0946 - accuracy: 0.9858 - val_loss: 0.5997 - val_accuracy: 0.7114
Epoch 9/25
14/14 [==============================] - 1s 74ms/step - loss: 0.0727 - accuracy: 0.9863 - val_loss: 0.5964 - val_accuracy: 0.7091
Epoch 10/25
14/14 [==============================] - 1s 77ms/step - loss: 0.0561 - accuracy: 0.9858 - val_loss: 0.6180 - val_accuracy: 0.7114
Epoch 11/25
14/14 [==============================] - 1s 82ms/step - loss: 0.0461 - accuracy: 0.9920 - val_loss: 0.6356 - val_accuracy: 0.6955
Epoch 12/25
14/14 [==============================] - 1s 74ms/step - loss: 0.0394 - accuracy: 0.9909 - val_los

In [ ]:
bestRNNTabulable = AverageStadisticsForModels(n, 'RNN Best Params', RNN_best_hyperparams, X_train, X_test, y_train, y_test)
print(tabulate(bestRNNTabulable, headers='keys', tablefmt='fancy_grid'))

Epoch 1/25
14/14 [==============================] - 3s 98ms/step - loss: 0.6954 - accuracy: 0.5392 - val_loss: 0.6804 - val_accuracy: 0.5886
Epoch 2/25
14/14 [==============================] - 1s 81ms/step - loss: 0.6598 - accuracy: 0.6809 - val_loss: 0.6614 - val_accuracy: 0.6568
Epoch 3/25
14/14 [==============================] - 1s 75ms/step - loss: 0.5973 - accuracy: 0.7577 - val_loss: 0.6381 - val_accuracy: 0.6136
Epoch 4/25
14/14 [==============================] - 1s 71ms/step - loss: 0.4891 - accuracy: 0.8914 - val_loss: 0.5837 - val_accuracy: 0.7364
Epoch 5/25
14/14 [==============================] - 1s 70ms/step - loss: 0.3874 - accuracy: 0.9204 - val_loss: 0.5363 - val_accuracy: 0.7659
Epoch 6/25
14/14 [==============================] - 1s 71ms/step - loss: 0.2750 - accuracy: 0.9482 - val_loss: 0.7428 - val_accuracy: 0.5773
Epoch 7/25
14/14 [==============================] - 2s 108ms/step - loss: 0.3490 - accuracy: 0.8544 - val_loss: 0.5192 - val_accuracy: 0.7477
Epoch 8/25
1

In [ ]:
LSTMTabulable = AverageStadisticsForModels(n, 'LSTM', LSTMFunction, X_train, X_test, y_train, y_test)
print(tabulate(LSTMTabulable, headers='keys', tablefmt='fancy_grid'))

Se han truncado las últimas 5000 líneas del flujo de salida.
14/14 [==============================] - 3s 229ms/step - loss: 0.2542 - accuracy: 0.9403 - val_loss: 0.4418 - val_accuracy: 0.8182
Epoch 7/25
14/14 [==============================] - 3s 229ms/step - loss: 0.1994 - accuracy: 0.9585 - val_loss: 0.4236 - val_accuracy: 0.8091
Epoch 8/25
14/14 [==============================] - 5s 396ms/step - loss: 0.1503 - accuracy: 0.9659 - val_loss: 0.4336 - val_accuracy: 0.8068
Epoch 9/25
14/14 [==============================] - 3s 233ms/step - loss: 0.1132 - accuracy: 0.9716 - val_loss: 0.4546 - val_accuracy: 0.8091
Epoch 10/25
14/14 [==============================] - 3s 238ms/step - loss: 0.0906 - accuracy: 0.9744 - val_loss: 0.4819 - val_accuracy: 0.7955
Epoch 11/25
14/14 [==============================] - 3s 241ms/step - loss: 0.0714 - accuracy: 0.9778 - val_loss: 0.5197 - val_accuracy: 0.8023
Epoch 12/25
14/14 [==============================] - 5s 388ms/step - loss: 0.0625 - accuracy: 0.

In [ ]:
bestLSTMTabulable = AverageStadisticsForModels(n, 'LSTM Best Params', LSTM_best_hyperparams, X_train, X_test, y_train, y_test)
print(tabulate(bestLSTMTabulable, headers='keys', tablefmt='fancy_grid'))

Se han truncado las últimas 5000 líneas del flujo de salida.
14/14 [==============================] - 3s 229ms/step - loss: 0.2151 - accuracy: 0.9494 - val_loss: 0.4790 - val_accuracy: 0.8045
Epoch 7/25
14/14 [==============================] - 3s 239ms/step - loss: 0.1512 - accuracy: 0.9539 - val_loss: 0.4966 - val_accuracy: 0.8136
Epoch 8/25
14/14 [==============================] - 5s 388ms/step - loss: 0.1091 - accuracy: 0.9699 - val_loss: 0.5004 - val_accuracy: 0.7773
Epoch 9/25
14/14 [==============================] - 3s 237ms/step - loss: 0.1033 - accuracy: 0.9738 - val_loss: 0.4738 - val_accuracy: 0.8045
Epoch 10/25
14/14 [==============================] - 3s 231ms/step - loss: 0.0848 - accuracy: 0.9767 - val_loss: 0.5182 - val_accuracy: 0.7909
Epoch 11/25
14/14 [==============================] - 3s 238ms/step - loss: 0.0592 - accuracy: 0.9807 - val_loss: 0.5978 - val_accuracy: 0.7909
Epoch 12/25
14/14 [==============================] - 5s 383ms/step - loss: 0.0531 - accuracy: 0.